<a href="https://colab.research.google.com/github/zjzsu2000/CMPE258/blob/master/assignment4/b)_3_ResNet_cifar10_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Graded Assignment 4 : CNNs
b) CNN basics : use pytorch and from scratch (not using existing architecture shipped models)  write basic LeNet (mnist) and VGG-16 model (cifar100) and and RESNET (only 8 layers - not 52)  architectures - run on gpu  (use .cuda instruction) - train and report all the stats

#  # b) 3_ResNet-cifar10_pytorch

## Resnet_8 Definition with PyTorch

In [0]:
import torch
import torch.nn as nn
from torch.nn import Module,Linear

In [0]:
class CV_block (Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(CV_block, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion*planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ID_block(Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(ID_block, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out




class ResNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(CV_block, 64, 1, stride=1)
        self.layer2 = self._make_layer(CV_block, 128, 1, stride=2)
        self.layer3 = self._make_layer(ID_block, 256, 1, stride=2)        
        self.linear = nn.Linear(256*4, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)        
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out






In [21]:
model = ResNet()
y = model(torch.randn(1,3,32,32))
print(y.size())

torch.Size([1, 10])


In [22]:
y

tensor([[-0.0159,  0.1592, -0.0531,  0.3048, -0.4558,  0.0619, -0.0231,  0.1900,
         -0.6838,  0.1320]], grad_fn=<AddmmBackward>)

In [23]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): CV_block(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
  )
  (layer2): Sequential(
  

### load data

In [0]:
import torch
import torch.nn as nn
import os,argparse
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
from torchvision import transforms


In [25]:
# use GPU
torch.manual_seed(42)
use_cuda = torch.cuda.is_available()
print(use_cuda)

device = torch.device("cuda" if use_cuda else "cpu")

device

True


device(type='cuda')

## train——data

In [26]:
transform_train = transforms.Compose([transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

Files already downloaded and verified


In [27]:
transform_test = transforms.Compose([transforms.ToTensor(),
                      transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),])

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

Files already downloaded and verified


In [0]:
classes = ('plane', 'car', 'bird', 'cat', 'deer','dog', 'frog', 'horse', 'ship', 'truck')

In [31]:
lr = 0.01

best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

# Model
print('==> Building model..')
net = ResNet().to(device)


if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

==> Building model..


In [0]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=lr,momentum=0.9, weight_decay=5e-4)

In [0]:
# Training
def training(epoch):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

        print(batch_idx, len(trainloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                     % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))


In [0]:
def testing(epoch):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            print(batch_idx, len(testloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                         % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))

    acc = 100.*correct/total
    if acc > best_acc:
        print('Saving..')
        state = {
            'net': net.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir('checkpoint'):
            os.mkdir('checkpoint')
        torch.save(state, './checkpoint/ckpt.pth')
        best_acc = acc


In [35]:
for epoch in range(0, 30):
    training(epoch)
    testing(epoch)

Streaming output truncated to the last 5000 lines.
34 100 Loss: 0.521 | Acc: 84.171% (2946/3500)
35 100 Loss: 0.520 | Acc: 84.250% (3033/3600)
36 100 Loss: 0.525 | Acc: 84.135% (3113/3700)
37 100 Loss: 0.526 | Acc: 84.026% (3193/3800)
38 100 Loss: 0.522 | Acc: 84.103% (3280/3900)
39 100 Loss: 0.521 | Acc: 84.125% (3365/4000)
40 100 Loss: 0.518 | Acc: 84.122% (3449/4100)
41 100 Loss: 0.520 | Acc: 84.143% (3534/4200)
42 100 Loss: 0.517 | Acc: 84.186% (3620/4300)
43 100 Loss: 0.515 | Acc: 84.182% (3704/4400)
44 100 Loss: 0.513 | Acc: 84.200% (3789/4500)
45 100 Loss: 0.514 | Acc: 84.174% (3872/4600)
46 100 Loss: 0.512 | Acc: 84.149% (3955/4700)
47 100 Loss: 0.515 | Acc: 84.167% (4040/4800)
48 100 Loss: 0.512 | Acc: 84.245% (4128/4900)
49 100 Loss: 0.516 | Acc: 84.120% (4206/5000)
50 100 Loss: 0.512 | Acc: 84.196% (4294/5100)
51 100 Loss: 0.512 | Acc: 84.135% (4375/5200)
52 100 Loss: 0.510 | Acc: 84.189% (4462/5300)
53 100 Loss: 0.507 | Acc: 84.278% (4551/5400)
54 100 Loss: 0.509 | Acc: 84.

Accuracy:86.72% 